# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
pip install evaluate scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Section imports
from datasets import load_dataset
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate

# Create variables and constants
FOUNDATION_MODEL = "gpt2"
tokenized_dataset = {}
splits = ["train", "test"]

# Downloading the date set
column_label = "verse_text"
dataset_name='poem_sentiment'
dataset = load_dataset(dataset_name, split='train').train_test_split(test_size=0.2, shuffle=True, seed=24)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [18]:
# Defined function to process tokenized inputs

def tokenized_inputs(inputs, tokenizer):
    prompt = inputs[column_label]
    tokenizer.truncation_side = "left"
    tokenized_items = tokenizer(
        prompt,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    return tokenized_items



# Defined function to evaluate accuracy of model
# https://huggingface.co/spaces/evaluate-metric/precision
def compute_metrics(eval_preds):
    metric = evaluate.load("precision")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="micro")


# Created Generic function to return the Trainer object
def get_trainer(trainer_model, dataset_name, metrics, num_epoch, output_dir, tokenizer):
    trained_model = Trainer(
        model=trainer_model,
        args=TrainingArguments(
            output_dir=output_dir,
            learning_rate=2e-5,
            per_device_train_batch_size=32,
            per_device_eval_batch_size=96,
            eval_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=num_epoch,
            weight_decay=0.01,
            load_best_model_at_end=True
        ),
        train_dataset=dataset_name["train"],
        eval_dataset=dataset_name["test"],
        tokenizer=tokenizer,
        compute_metrics=metrics,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    )
    return trained_model


# Define Tokenizer function
def get_tokenizer(modal_name):
    tokenizer = AutoTokenizer.from_pretrained(modal_name)
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

In [19]:
# Configuring the tokenizer and adding padding tokens function
tokenizer_model = get_tokenizer(FOUNDATION_MODEL)

# Tokenize inputs
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenized_inputs(x, tokenizer_model), batched=True
    )

print("tokenized_dataset", tokenized_dataset["train"])

tokenized_dataset Dataset({
    features: ['id', 'verse_text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 713
})


In [20]:
# Create Label objects to pass down to ouw model
id2label = {0: "negative", 1: "positive", 2: "no impact", 3: "mixed"}
label2id ={"negative": 0, "positive": 1, "no impact": 2, "mixed": 3}

# Loading the foundation model with the correct number of labels and identifiers
model = AutoModelForSequenceClassification.from_pretrained(
    FOUNDATION_MODEL,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

model.config.pad_token_id = model.config.eos_token_id
model.resize_token_embeddings(len(tokenizer_model))
model.to(device)

# Created Trainer Object and loaded the original model
gpt2_model_trainer = get_trainer(
    trainer_model=model,
    dataset_name=tokenized_dataset,
    metrics=compute_metrics,
    num_epoch=1,
    output_dir="./",
    tokenizer=tokenizer_model
)
print(f'Evaluating foundation model...')
original_model_eval = gpt2_model_trainer.evaluate()
print(original_model_eval)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluating foundation model...


100%|██████████| 2/2 [00:01<00:00,  1.01it/s]

{'eval_loss': 3.0786139965057373, 'eval_precision': 0.22346368715083798, 'eval_runtime': 2.0498, 'eval_samples_per_second': 87.325, 'eval_steps_per_second': 0.976}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [21]:
from peft import LoraConfig, get_peft_model, TaskType

In [55]:
# Configure LoRa settings
targets = ["c_proj", "c_fc", "c_attn"]
config = LoraConfig(
    r=32,
    lora_alpha=96,
    lora_dropout=0.01,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    target_modules=targets,
    fan_in_fan_out=True
)
# Create Peft Model
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 4,721,664 || all params: 129,164,544 || trainable%: 3.6555


In [56]:
# Define Train Object Using Lora Model from Peft
lora_trainer = get_trainer(
    trainer_model=lora_model, 
    dataset_name=tokenized_dataset, 
    metrics=compute_metrics, 
    num_epoch=6, 
    output_dir="./gpt-lora",
    tokenizer=tokenizer_model
)
lora_trainer.train()
# Print lora_trainer evaluation
eval_lora = lora_trainer.evaluate()
print(eval_lora)

 17%|█▋        | 23/138 [00:04<00:10, 10.97it/s]c:\Users\freda\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 2.0250444412231445, 'eval_precision': 0.2905027932960894, 'eval_runtime': 1.8441, 'eval_samples_per_second': 97.064, 'eval_steps_per_second': 1.085, 'epoch': 1.0}


 33%|███▎      | 46/138 [00:08<00:08, 10.23it/s]c:\Users\freda\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'eval_loss': 1.6405073404312134, 'eval_precision': 0.5307262569832403, 'eval_runtime': 2.0497, 'eval_samples_per_second': 87.331, 'eval_steps_per_second': 0.976, 'epoch': 2.0}


 50%|█████     | 69/138 [00:13<00:06, 10.38it/s]c:\Users\freda\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

{'eval_loss': 1.3446208238601685, 'eval_precision': 0.5810055865921788, 'eval_runtime': 2.53, 'eval_samples_per_second': 70.752, 'eval_steps_per_second': 0.791, 'epoch': 6.0}


100%|██████████| 138/138 [00:28<00:00,  4.85it/s]


{'train_runtime': 28.4812, 'train_samples_per_second': 150.204, 'train_steps_per_second': 4.845, 'train_loss': 1.3707465572633606, 'epoch': 6.0}


100%|██████████| 2/2 [00:02<00:00,  1.05s/it]

{'eval_loss': 1.3446208238601685, 'eval_precision': 0.5810055865921788, 'eval_runtime': 2.3706, 'eval_samples_per_second': 75.507, 'eval_steps_per_second': 0.844, 'epoch': 6.0}


In [57]:
lora_model.save_pretrained("gpt-lora-trained")

c:\Users\freda\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [58]:
from peft import AutoPeftModelForSequenceClassification
import torch

In [59]:
# Loading saved PEFT model from directory
lora_peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./gpt-lora-trained", num_labels=4)
lora_peft_model.config.pad_token_id = lora_peft_model.config.eos_token_id
lora_peft_model.resize_token_embeddings(len(tokenizer_model))
lora_peft_model.to(device)

# Load Trained Tokenizer
lora_model_trainer = get_trainer(
    trainer_model=lora_peft_model,
    dataset_name=tokenized_dataset,
    metrics=compute_metrics,
    num_epoch=10,
    output_dir="./",
    tokenizer=tokenizer_model
)
lora_model_eval = lora_model_trainer.evaluate()
print(lora_model_eval)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 2/2 [00:02<00:00,  1.13s/it]

{'eval_loss': 1.6635046005249023, 'eval_precision': 0.5195530726256983, 'eval_runtime': 2.3708, 'eval_samples_per_second': 75.503, 'eval_steps_per_second': 0.844}


In [60]:
# Starting Inference
input_text = "in prosperous days. like a dim, waning lamp"
model_inputs = tokenizer_model(input_text, return_tensors="pt").to(device)

In [61]:
with torch.no_grad():
    outputs = lora_peft_model(**model_inputs)
    results = torch.argmax(outputs.logits, dim=-1)
# Converting Prediction to Label
predicted_label = id2label[results.item()]

In [62]:
# Input Text with Predicted outout
print(f"Input: {input_text}\n Predicted label: {predicted_label}")

Input: in prosperous days. like a dim, waning lamp
 Predicted label: no impact


In [63]:
# comparing Evaluation Results
print(f'Original model eval: {original_model_eval} \nLora Model Eval: {lora_model_eval}')

Original model eval: {'eval_loss': 3.0786139965057373, 'eval_precision': 0.22346368715083798, 'eval_runtime': 2.0498, 'eval_samples_per_second': 87.325, 'eval_steps_per_second': 0.976} 
Lora Model Eval: {'eval_loss': 1.6635046005249023, 'eval_precision': 0.5195530726256983, 'eval_runtime': 2.3708, 'eval_samples_per_second': 75.503, 'eval_steps_per_second': 0.844}
